In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential , load_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [28]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [29]:
cd /content/gdrive/MyDrive

/content/gdrive/MyDrive


In [ ]:
import os
import shutil

# Define the path to the original dataset
data_path = '/content/gdrive/MyDrive/AICapstoneProjectData/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone'
# Define the path to the output directory
output_path = './dataset'

# Define the ratios for train/val/test splits
train_ratio = 0.8
test_ratio = 0.2

# Create the output directories if they do not exist
if not os.path.exists(output_path):
    os.makedirs(output_path)
for dir_name in ['train', 'test']:
    dir_path = os.path.join(output_path, dir_name)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    for class_name in ['Cyst','Normal','Stone','Tumor']:
        class_path = os.path.join(dir_path, class_name)
        if not os.path.exists(class_path):
            os.makedirs(class_path)

# Loop over each class and copy files to train/val/test directories
for class_name in ['Cyst', 'Normal', 'Stone', 'Tumor']:
    class_path = os.path.join(data_path, class_name)
    files = os.listdir(class_path)
    num_files = len(files)
    num_train = int(num_files * train_ratio)
    num_test = num_files - num_train
    for i, file_name in enumerate(files):
        src_path = os.path.join(class_path, file_name)
        if i < num_train:
            dst_path = os.path.join(output_path, 'train', class_name, file_name)
        else:
            dst_path = os.path.join(output_path, 'test', class_name, file_name)
        shutil.copy(src_path, dst_path)

In [44]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

# Set up the image data generators
train_path = './dataset/train'
test_path = './dataset/test'
img_size = (224,224)
batch_size = 10
test_batch_size = 10
test_steps = 1

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8,1.2],
    preprocessing_function=preprocess_input,
)
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    directory=train_path,
    target_size=(224, 224),
    class_mode='categorical',
    shuffle=True,
    batch_size=batch_size
)
test_generator = test_datagen.flow_from_directory(
    directory=test_path,
    target_size=(224, 224),
    class_mode='categorical',
    shuffle=False,
    batch_size=test_batch_size
)


Found 9955 images belonging to 4 classes.
Found 2492 images belonging to 4 classes.


In [45]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import accuracy_score, confusion_matrix

# Load the data and preprocess it
train_data = train_generator[0][0]
train_labels = train_generator[0][1]
test_data = test_generator[0][0]
test_labels = test_generator[0][1]

img_height = 224
img_width = 224
num_classes =4
# Define the CNN architecture
model = tf.keras.Sequential([
  layers.Conv2D(32, kernel_size=3, activation='relu', input_shape=(img_height, img_width, 3)),
  layers.MaxPooling2D(pool_size=(2, 2)),
  layers.Conv2D(64, kernel_size=3, activation='relu'),
  layers.MaxPooling2D(pool_size=(2, 2)),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_data, train_labels, epochs=100, validation_split=0.2)

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_data, test_labels)
print('Test accuracy:', test_acc)

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 1.3703 - accuracy: 0.3750 - val_loss: 23.8305 - val_accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 2s 2s/step - loss: 5.9378 - accuracy: 0.5000 - val_loss: 12.2882 - val_accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 2s 2s/step - loss: 0.1172 - accuracy: 0.8750 - val_loss: 12.0011 - val_accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 1s 1s/step - loss: 0.6898 - accuracy: 0.7500 - val_loss: 9.6368 - val_accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 1s 897ms/step - loss: 0.0122 - accuracy: 1.0000 - val_loss: 8.2938 - val_accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 1s 885ms/step - loss: 0.0058 - accuracy: 1.0000 - val_loss: 7.3408 - val_accuracy: 0.0000e+00
Epoch 7/100
1/1 [==============================] - 1s 890ms/step - loss: 0.0070 - accuracy: 1.0000 - val_loss: 6.8686 - val_accuracy: 0.000

In [50]:
train_loss , train_acc = model.evaluate(train_data, train_labels)
print('Train accuracy:', train_acc*100)
print('Test accuracy:', test_acc*100)

1/1 [==============================] - 0s 327ms/step - loss: 2.0895 - accuracy: 0.8000
Train accuracy: 80.0000011920929
Test accuracy: 80.0000011920929


In [51]:
# Get the predictions for the test data
test_preds = model.predict(test_data)
test_preds = np.argmax(test_preds, axis=1)

# Get the true labels for the test data
test_true = np.argmax(test_labels, axis=1)

# Calculate the confusion matrix for the test data
conf_matrix_test = confusion_matrix(test_true, test_preds)

print("Training accuracy: {:.2f}%".format(train_acc*100))
print("Testing accuracy: {:.2f}%".format(test_acc*100))
print("Testing confusion matrix:")
print(conf_matrix_test)

1/1 [==============================] - 0s 324ms/step
Training accuracy: 80.00%
Testing accuracy: 80.00%
Testing confusion matrix:
[[8 2]
 [0 0]]


In [52]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
cr = classification_report(test_true, test_preds)
print("Classification report:")
print(cr)

Classification report:
              precision    recall  f1-score   support

           0       1.00      0.80      0.89        10
           1       0.00      0.00      0.00         0

    accuracy                           0.80        10
   macro avg       0.50      0.40      0.44        10
weighted avg       1.00      0.80      0.89        10



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
